# 0. Instalacion de utilitarios Pyspark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.3.4/spark-2.3.4-bin-hadoop2.7.tgz
!tar xf spark-2.3.4-bin-hadoop2.7.tgz

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz


In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"


In [ ]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 45kB/s 
     |████████████████████████████████| 204kB 46.7MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=c9ed2064f7a916d9184b3c8382e1fa017c1c1fdee691d40d7a4d569e6ea6747c
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

# 1. Lectura del origen datos

In [ ]:
# Conexion al drive
#---------------------------
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
ls

drive/        spark-2.3.4-bin-hadoop2.7/     spark-2.4.4-bin-hadoop2.7/
sample_data/  spark-2.3.4-bin-hadoop2.7.tgz  spark-2.4.4-bin-hadoop2.7.tgz


#2.  Cargando los datos en formato Spark

In [ ]:
DS_peliculas_rating = spark.read.csv(
    '/content/drive/My Drive/PySpark Machine Learning en plataforma Big Data/Training R ML en Casos de Negocios/8_Recomendacion_comercio/Comercio_tipo_rappi.csv', 
    sep=';', 
    header=True, 
    inferSchema=True
)
# 2.1 Revision de formatos 
DS_peliculas_rating.printSchema()

root
 |-- userID: string (nullable = true)
 |-- userID_cod: integer (nullable = true)
 |-- placeID: integer (nullable = true)
 |-- Nombre_comerce: string (nullable = true)
 |-- Distrito: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- food_rating: integer (nullable = true)
 |-- service_rating: integer (nullable = true)
 |-- rating_total: integer (nullable = true)



In [ ]:
# Dimensiona de base
print((DS_peliculas_rating.count(),len(DS_peliculas_rating.columns)))

(50766, 9)


In [ ]:
# 2.2 Revisiones  
DS_peliculas_rating.show(5)

+------+----------+-------+--------------------+---------------+------+-----------+--------------+------------+
|userID|userID_cod|placeID|      Nombre_comerce|       Distrito|rating|food_rating|service_rating|rating_total|
+------+----------+-------+--------------------+---------------+------+-----------+--------------+------------+
| U1077|         1| 135085|Tortas Locas Hipo...|San Luis Potosi|     2|          2|             2|           6|
| U1077|         1| 135038|Restaurant la Cha...|San Luis Potosi|     2|          2|             1|           5|
| U1077|         1| 132825|     puesto de tacos|         s.l.p.|     2|          2|             2|           6|
| U1077|         1| 135060|Restaurante Maris...|San Luis Potosi|     1|          2|             2|           5|
| U1068|         2| 135104|                vips|              0|     1|          1|             2|           4|
+------+----------+-------+--------------------+---------------+------+-----------+--------------+------

In [ ]:
# 2.3 Validacion de carga del total de registros
print(DS_peliculas_rating.count())

50766


In [ ]:
# 2.4 Exploracion inicial
DS_peliculas_rating.describe().show()

+-------+------+------------------+------------------+--------------------+---------+------------------+------------------+------------------+-----------------+
|summary|userID|        userID_cod|           placeID|      Nombre_comerce| Distrito|            rating|       food_rating|    service_rating|     rating_total|
+-------+------+------------------+------------------+--------------------+---------+------------------+------------------+------------------+-----------------+
|  count| 50766|             50766|             50766|               50766|    50766|             50766|             50766|             50766|            50766|
|   mean|  null| 67.34026710790687|  134254.559390143|                null|      0.0|1.2218216916834101| 1.244277666154513| 1.109856990899421|3.575956348737344|
| stddev|  null|32.640689951647815|1079.9216692471784|                null|      0.0|0.7706200957794772|0.7863420037824467|0.7915679876580894|2.083855254035862|
|    min| U1001|                 1

In [ ]:
# 2.5 Los clientes que más vistaron la plataforma de videos
DS_peliculas_rating.groupBy('userId').count().orderBy('count',ascending=False).show(10,False)

+------+-----+
|userId|count|
+------+-----+
|U1106 |1198 |
|U1071 |949  |
|U1024 |933  |
|U1016 |897  |
|U1036 |876  |
|U1104 |868  |
|U1009 |759  |
|U1037 |755  |
|U1084 |730  |
|U1132 |730  |
+------+-----+
only showing top 10 rows



In [ ]:
# 2.6  Las clientes que menos vieron las peliculas
DS_peliculas_rating.groupBy('userId').count().orderBy('count',ascending=True).show(10,False)

+------+-----+
|userId|count|
+------+-----+
|U1011 |57   |
|U1039 |57   |
|U1129 |76   |
|U1130 |76   |
|U1065 |95   |
|U1043 |95   |
|U1138 |114  |
|U1102 |114  |
|U1077 |117  |
|U1131 |120  |
+------+-----+
only showing top 10 rows



In [ ]:
# 2.7  Las peliculas con mayor numero de visualizaciones
DS_peliculas_rating.groupBy('Nombre_comerce').count().orderBy('count',ascending=False).show(10,False)

+----------------------------------+-----+
|Nombre_comerce                    |count|
+----------------------------------+-----+
|Tortas Locas Hipocampo            |1641 |
|Cafeteria y Restaurant El Pacifico|1466 |
|puesto de tacos                   |1387 |
|La Cantina Restaurante            |1202 |
|Restaurant la Chalita             |1200 |
|Gorditas Doa Gloria               |1176 |
|Restaurante Marisco Sam           |1017 |
|Restaurant Oriental Express       |955  |
|Restaurante El Cielo Potosino     |945  |
|Luna Cafe                         |836  |
+----------------------------------+-----+
only showing top 10 rows



In [ ]:
# Peliculas con menos numero de visualizaciones 
DS_peliculas_rating.groupBy('Nombre_comerce').count().orderBy('count',ascending=True).show(10,False)

+-------------------------------------+-----+
|Nombre_comerce                       |count|
+-------------------------------------+-----+
|emilianos                            |99   |
|Carnitas Mata  Calle 16 de Septiembre|100  |
|cafe ambar                           |100  |
|la perica hamburguesa                |102  |
|tortas hawai                         |109  |
|puesto de gorditas                   |109  |
|tacos los volcanes                   |118  |
|churchs                              |126  |
|TACOS EL GUERO                       |133  |
|Arrachela Grill                      |135  |
+-------------------------------------+-----+
only showing top 10 rows



# 3. FEature Engineering: Tansforma pelicula a numero



In [ ]:
# Importando el codifocador
from pyspark.ml.feature import StringIndexer,IndexToString

In [ ]:
# Configurando la Numerzación los titulos de las peliculas
# title_new = La codificación de las peliculas
stringIndexer = StringIndexer(inputCol="Nombre_comerce", outputCol="Nombre_comerce_id")

In [ ]:
#Aplicando la numerización de peliculas
model = stringIndexer.fit(DS_peliculas_rating)

In [ ]:
#Creando un nuevo Dataframe con el indexado creado


# 4. Dividiendo la data en muestra de entrenamiento y validacion


In [ ]:
DS_peliculas_rating = DS_peliculas_rating.drop('rating', 'food_rating', 'service_rating')
DS_peliculas_rating = DS_peliculas_rating.withColumn('rating', DS_peliculas_rating.rating_total)

In [ ]:
DS_peliculas_rating.show(5)

+------+----------+-------+--------------------+---------------+------------+------+
|userID|userID_cod|placeID|      Nombre_comerce|       Distrito|rating_total|rating|
+------+----------+-------+--------------------+---------------+------------+------+
| U1077|         1| 135085|Tortas Locas Hipo...|San Luis Potosi|           6|     6|
| U1077|         1| 135038|Restaurant la Cha...|San Luis Potosi|           5|     5|
| U1077|         1| 132825|     puesto de tacos|         s.l.p.|           6|     6|
| U1077|         1| 135060|Restaurante Maris...|San Luis Potosi|           5|     5|
| U1068|         2| 135104|                vips|              0|           4|     4|
+------+----------+-------+--------------------+---------------+------------+------+
only showing top 5 rows



In [ ]:
indexed = model.transform(DS_peliculas_rating)
indexed.show(10)
indexed.groupBy('Nombre_comerce_id').count().orderBy('count',ascending=False).show(10,False)

+------+----------+-------+--------------------+---------------+------------+------+-----------------+
|userID|userID_cod|placeID|      Nombre_comerce|       Distrito|rating_total|rating|Nombre_comerce_id|
+------+----------+-------+--------------------+---------------+------------+------+-----------------+
| U1077|         1| 135085|Tortas Locas Hipo...|San Luis Potosi|           6|     6|              0.0|
| U1077|         1| 135038|Restaurant la Cha...|San Luis Potosi|           5|     5|              4.0|
| U1077|         1| 132825|     puesto de tacos|         s.l.p.|           6|     6|              2.0|
| U1077|         1| 135060|Restaurante Maris...|San Luis Potosi|           5|     5|              6.0|
| U1068|         2| 135104|                vips|              0|           4|     4|             80.0|
| U1068|         2| 132740|Carreton de Flaut...|Ciudad Victoria|           0|     0|             72.0|
| U1068|         2| 132663|           tacos abi|       victoria|         

In [ ]:
#4.1  Data de entrenamiento y validacion
train, test=indexed.randomSplit([0.8, 0.2], seed=1234)

In [ ]:
# Nro de entrenamiento
train.count()

40541

In [ ]:
# Nro de Validacion
test.count()

10225

In [ ]:
train.show(3)

+------+----------+-------+---------------+--------+------------+------+-----------------+
|userID|userID_cod|placeID| Nombre_comerce|Distrito|rating_total|rating|Nombre_comerce_id|
+------+----------+-------+---------------+--------+------------+------+-----------------+
| U1001|        39| 132825|puesto de tacos|  s.l.p.|           5|     5|              2.0|
| U1001|        39| 132825|puesto de tacos|  s.l.p.|           5|     5|              2.0|
| U1001|        39| 132825|puesto de tacos|  s.l.p.|           5|     5|              2.0|
+------+----------+-------+---------------+--------+------------+------+-----------------+
only showing top 3 rows



# 5. Entrenando el Modelo


In [ ]:
#Importando el modelo Recomendador ALS
from pyspark.sql.functions import *
from pyspark.ml.recommendation import ALS


In [ ]:
#Configuracion del Entrenamiento del modelo Recomendador usando la muetsra de entrenamiento
rec=ALS(
    maxIter=25,
    regParam=0.025,
    userCol='userID_cod',
    itemCol='Nombre_comerce_id',
    ratingCol='rating',
    nonnegative=True,
    coldStartStrategy="drop",
    implicitPrefs=False, 
    seed=1234
  )

# maxIter: El maximo de iteraciones
# regParam: Parametro de REgularización
# nonnegative: Especificar si se utiliza una restriccion negativa en los Minimos cuadrados
# coldStartStrategy: Parametro que elimina cualquier cliente que no tenga informacion
# implicitPrefs: Si es implicito hace referencia a feedback no registrado voluntariamente por el cliente

In [ ]:
#Entrenando el modelo
rec_model = rec.fit(train)

In [ ]:
#Haciendo las predicciones en la data de test 
predicted_ratings = rec_model.transform(test)

In [ ]:
#Revisando los pronostico
predicted_ratings.printSchema()

root
 |-- userID: string (nullable = true)
 |-- userID_cod: integer (nullable = true)
 |-- placeID: integer (nullable = true)
 |-- Nombre_comerce: string (nullable = true)
 |-- Distrito: string (nullable = true)
 |-- rating_total: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- Nombre_comerce_id: double (nullable = false)
 |-- prediction: float (nullable = false)



In [ ]:
# Revision rapida de los pronosticos
predicted_ratings.orderBy(rand()).show(10)

+------+----------+-------+--------------------+---------------+------------+------+-----------------+----------+
|userID|userID_cod|placeID|      Nombre_comerce|       Distrito|rating_total|rating|Nombre_comerce_id|prediction|
+------+----------+-------+--------------------+---------------+------------+------+-----------------+----------+
| U1110|        87| 134999|     Kiku Cuernavaca|     Cuernavaca|           6|     6|             67.0|  5.986145|
| U1022|        51| 135033|Restaurant El Mul...|San Luis Potosi|           4|     4|             90.0| 3.9748328|
| U1135|        95| 135060|Restaurante Maris...|San Luis Potosi|           0|     0|              6.0|       0.0|
| U1045|        76| 135032|Cafeteria y Resta...|San Luis Potosi|           0|     0|              1.0|0.25708318|
| U1116|        53| 135059|Restaurant Bar Ha...|San Luis Potosi|           4|     4|             49.0|  4.160031|
| U1102|       101| 132870|Tortas y hamburgu...|San Luis Potosi|           4|     4|    

In [ ]:
# Generando los top 10 peliculas sugeridas por cada cliente
userRecs = rec_model.recommendForAllUsers(10)

In [ ]:
userRecs.show(5, False)

+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userID_cod|recommendations                                                                                                                                                          |
+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|31        |[[94, 5.002146], [119, 4.9522953], [92, 4.9440827], [43, 4.785837], [4, 4.305575], [25, 4.2629194], [76, 4.065121], [68, 4.050144], [18, 3.9866621], [71, 3.9593832]]    |
|85        |[[77, 5.956505], [14, 4.765882], [105, 4.4923162], [73, 4.483544], [86, 4.229867], [7, 4.2258863], [21, 4.199602], [37, 3.9781835], [65, 3.9648116], [62, 3.9604897]]    |
|137       |[[92, 6.4969544], [25, 6.0120206], [4, 5.972985], [35, 5.7718644], [40, 5

# 5. Evaluación del poder predictivo del modelo


In [ ]:
# Creando la columna diferencia del pronostico
predicted_ratings_witherr = predicted_ratings.withColumn(
    'err', abs(predicted_ratings.prediction - predicted_ratings.rating)
)

predicted_ratings_witherr.show()

+------+----------+-------+---------------+--------+------------+------+-----------------+----------+---------+
|userID|userID_cod|placeID| Nombre_comerce|Distrito|rating_total|rating|Nombre_comerce_id|prediction|      err|
+------+----------+-------+---------------+--------+------------+------+-----------------+----------+---------+
| U1062|        93| 132872|Pizzeria Julios|   s.l.p|           0|     0|             31.0|       0.0|      0.0|
| U1062|        93| 132872|Pizzeria Julios|   s.l.p|           0|     0|             31.0|       0.0|      0.0|
| U1062|        93| 132872|Pizzeria Julios|   s.l.p|           0|     0|             31.0|       0.0|      0.0|
| U1062|        93| 132872|Pizzeria Julios|   s.l.p|           0|     0|             31.0|       0.0|      0.0|
| U1062|        93| 132872|Pizzeria Julios|   s.l.p|           0|     0|             31.0|       0.0|      0.0|
| U1062|        93| 132872|Pizzeria Julios|   s.l.p|           0|     0|             31.0|       0.0|   

In [ ]:
# Distribución de las valoraciones
predicted_ratings.groupBy('rating').count().orderBy('rating',ascending=True).show()

+------+-----+
|rating|count|
+------+-----+
|     0| 1612|
|     1|  326|
|     2|  747|
|     3| 2016|
|     4| 1519|
|     5| 1278|
|     6| 2727|
+------+-----+



In [ ]:
# Error promedio por rating de valoracion
predicted_ratings_witherr.groupBy('rating').agg({'err':'mean'}).orderBy('rating',ascending=True).show()

+------+-------------------+
|rating|           avg(err)|
+------+-------------------+
|     0|0.11545983166798571|
|     1|0.12181953071085222|
|     2|0.10642437124188486|
|     3| 0.0716917235699911|
|     4| 0.0726899568621778|
|     5| 0.0808926456970788|
|     6| 0.0804295004791159|
+------+-------------------+



In [ ]:
# Error promedio por rating de valoracion
predicted_ratings_witherr.groupBy('rating').agg({'err':'std'}).orderBy('rating',ascending=True).show()

+------+-------------------+
|rating|        stddev(err)|
+------+-------------------+
|     0|0.17001643611396375|
|     1|0.11162416989113194|
|     2|0.09136079152762806|
|     3|0.07643020301488447|
|     4|0.08190661843351289|
|     5|0.07848069842616054|
|     6|0.08187443500718211|
+------+-------------------+



In [ ]:
#Importando el evaluador de Regresion RMSE 
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
#Evluando la presición del pronostico
evaluator=RegressionEvaluator(metricName='rmse',predictionCol='prediction',labelCol='rating')

In [ ]:
#Evluando el error de pronostico global
rmse=evaluator.evaluate(predicted_ratings)

In [ ]:
#Imprimiento el error
print(rmse)

0.13418804205726628


# 6. Recomendandor de peliculas, ya listo!

In [ ]:
#Listado de peliculas unicas
unique_movies = indexed.select('title_new').distinct()

In [ ]:
#Total de peliculas
unique_movies.count()

1664

In [ ]:
#Asignando un alias {a} la lista unica de peliculas
a = unique_movies.alias('a')

In [ ]:
# Eligiendo un cliente
user_id=100

# Creando una tabla que contiene una pelicula por usuario
watched_movies=indexed.filter(indexed['userId'] == user_id).select('title_new').distinct()

#Numero de peliculas ya evaluadas 
watched_movies.count()

59

In [ ]:
# Asignando un alias  {b} ha peliculas que el usuario ya vio de la tabla
b=watched_movies.alias('b')

In [ ]:
#Cruzando ambas tablas
total_movies = a.join(b, a.title_new == b.title_new,how='left')

In [ ]:
total_movies.show(10,False)

+---------+---------+
|title_new|title_new|
+---------+---------+
|558.0    |null     |
|305.0    |null     |
|299.0    |null     |
|596.0    |null     |
|769.0    |null     |
|934.0    |null     |
|496.0    |null     |
|1051.0   |null     |
|692.0    |null     |
|810.0    |null     |
+---------+---------+
only showing top 10 rows



In [ ]:
# Seleccionando peliculas que el cliente aun no ha mirado
remaining_movies=total_movies.where(col("b.title_new").isNull()).select(a.title_new).distinct()

In [ ]:
remaining_movies.show()

+---------+
|title_new|
+---------+
|    558.0|
|    305.0|
|    299.0|
|    596.0|
|    769.0|
|    934.0|
|    496.0|
|   1051.0|
|    692.0|
|    810.0|
|    720.0|
|    782.0|
|    184.0|
|    147.0|
|    576.0|
|    170.0|
|   1369.0|
|   1587.0|
|    608.0|
|    735.0|
+---------+
only showing top 20 rows



In [ ]:
# Numero de peliculas que los usuario aun no lo ven 
remaining_movies.count()

1605

In [ ]:
# Agregando una nueva columna del usuario 
remaining_movies=remaining_movies.withColumn("userId",lit(int(user_id)))

In [ ]:
remaining_movies.show(10,False)

+---------+------+
|title_new|userId|
+---------+------+
|558.0    |100   |
|305.0    |100   |
|299.0    |100   |
|596.0    |100   |
|769.0    |100   |
|934.0    |100   |
|496.0    |100   |
|1051.0   |100   |
|692.0    |100   |
|810.0    |100   |
+---------+------+
only showing top 10 rows



In [ ]:
# Haciendo la recomendacion de peliculas utilizando el modelo recomendador ALS y seleccionado solo las mejores top {n} peliculas
recommendations=rec_model.transform(remaining_movies).orderBy('prediction',ascending=False)

In [ ]:
recommendations.show(5,False)

+---------+------+----------+
|title_new|userId|prediction|
+---------+------+----------+
|1066.0   |100   |6.513842  |
|930.0    |100   |5.687636  |
|923.0    |100   |5.469397  |
|1239.0   |100   |5.418679  |
|869.0    |100   |5.2863975 |
+---------+------+----------+
only showing top 5 rows



In [ ]:
# Consiguiendo los de las peliculas sugeridas
movie_title = IndexToString(inputCol="title_new", outputCol="title",labels=model.labels)

final_recommendations=movie_title.transform(recommendations)

In [ ]:
final_recommendations.show(10,False)

+---------+------+----------+-------------------------------------------------------------------------+
|title_new|userId|prediction|title                                                                    |
+---------+------+----------+-------------------------------------------------------------------------+
|1066.0   |100   |6.513842  |Alphaville (1965)                                                        |
|930.0    |100   |5.687636  |Ma vie en rose (My Life in Pink) (1997)                                  |
|923.0    |100   |5.469397  |Indian Summer (1996)                                                     |
|1239.0   |100   |5.418679  |Swept from the Sea (1997)                                                |
|869.0    |100   |5.2863975 |Burnt By the Sun (1994)                                                  |
|840.0    |100   |5.2617555 |Ghost in the Shell (Kokaku kidotai) (1995)                               |
|1121.0   |100   |5.0466433 |Crooklyn (1994)                    

# 7. Construyendo las funcion que sugiera una pelicula recomendada segun el modelo recomendador

In [ ]:
# Funcion para recomendar las top n peliculas para cualquier usuario
def top_movies(user_id,n):
    """
    This function returns the top 'n' movies that user has not seen yet but might like 
    
    """
    # Asignación de una lista de peliculas unicas a
    a = unique_movies.alias('a')
    
    # Creando otra tabla que contiene las peliculas que ya vio el usuario
    watched_movies=indexed.filter(indexed['userId'] == user_id).select('title_new')
    
    # Asignanción de tabla de peliculas b de peliculas ya vistas 
    b=watched_movies.alias('b')
    
    # Cruce de ambas tablas
    total_movies = a.join(b, a.title_new == b.title_new,how='left')
    
    # Seleccionando peliculas que el usuario no ha visto aun 
    remaining_movies=total_movies.where(col("b.title_new").isNull()).select(a.title_new).distinct()
        
    #Agregando una nueva columan del cliente  
    remaining_movies=remaining_movies.withColumn("userId",lit(int(user_id)))
    
    # REalizando la recomendnado de peliculas mas adecuada utilizando el Sistema de recomendador del modelo ALS y seleccionando las mejores peliculas
    recommendations=rec_model.transform(remaining_movies).orderBy('prediction',ascending=False).limit(n)

    # Agregando los titulos de las peliculas en texto   
    movie_title = IndexToString(inputCol="title_new", outputCol="title",labels=model.labels)
    final_recommendations=movie_title.transform(recommendations)
    
    # Entregando los titutlos recomendado por usuario
    return final_recommendations.show(n,False)


In [ ]:
# REvision del clientes y las 10 peliculas sugeridas para el cliente
top_movies(123,15)

+---------+------+----------+-------------------------------------------------------------------------+
|title_new|userId|prediction|title                                                                    |
+---------+------+----------+-------------------------------------------------------------------------+
|1358.0   |123   |6.666241  |Angel Baby (1995)                                                        |
|1121.0   |123   |6.456703  |Crooklyn (1994)                                                          |
|910.0    |123   |5.9415126 |Mis<U+008E>rables, Les (1995)                                            |
|739.0    |123   |5.817608  |Night on Earth (1991)                                                    |
|1184.0   |123   |5.6485686 |Maybe, Maybe Not (Bewegte Mann, Der) (1994)                              |
|1182.0   |123   |5.599687  |Kaspar Hauser (1993)                                                     |
|495.0    |123   |5.541688  |In the Company of Men (1997)       

In [ ]:
# El cliente 450 que revisa Star Wars
DS_peliculas_rating.filter((DS_peliculas_rating['rating']>4)&(DS_peliculas_rating['userId']=='450')).show()

+------+--------------------+------+
|userId|               title|rating|
+------+--------------------+------+
|   450|L.A. Confidential...|     5|
|   450|Dr. Strangelove o...|     5|
|   450|Hunt for Red Octo...|     5|
|   450|Age of Innocence,...|     5|
|   450|     Die Hard (1988)|     5|
|   450|Right Stuff, The ...|     5|
|   450|Sleepless in Seat...|     5|
|   450|   Sting, The (1973)|     5|
|   450|  Rear Window (1954)|     5|
|   450|Dead Poets Societ...|     5|
|   450|Sound of Music, T...|     5|
|   450|E.T. the Extra-Te...|     5|
|   450|Brothers McMullen...|     5|
|   450|To Kill a Mocking...|     5|
|   450|   Old Yeller (1957)|     5|
|   450|Jerry Maguire (1996)|     5|
|   450|     Backbeat (1993)|     5|
|   450|  Taxi Driver (1976)|     5|
|   450|Young Frankenstei...|     5|
|   450|It's a Wonderful ...|     5|
+------+--------------------+------+
only showing top 20 rows



In [ ]:
# REvision del clientes y las 10 peliculas sugeridas para el cliente
top_movies(450,15)

+---------+------+----------+----------------------------------+
|title_new|userId|prediction|title                             |
+---------+------+----------+----------------------------------+
|944.0    |450   |5.4597893 |Surviving Picasso (1996)          |
|1072.0   |450   |5.440328  |Radioland Murders (1994)          |
|1358.0   |450   |5.2606144 |Angel Baby (1995)                 |
|1271.0   |450   |5.168796  |Whole Wide World, The (1996)      |
|1266.0   |450   |5.114764  |Promesse, La (1996)               |
|923.0    |450   |5.1098905 |Indian Summer (1996)              |
|1023.0   |450   |5.100616  |Gay Divorcee, The (1934)          |
|581.0    |450   |5.0989923 |Koyaanisqatsi (1983)              |
|1195.0   |450   |5.0697556 |Pather Panchali (1955)            |
|988.0    |450   |5.066791  |Ruling Class, The (1972)          |
|997.0    |450   |5.0588284 |Selena (1997)                     |
|1450.0   |450   |5.0580397 |C<U+008E>r<U+008E>monie, La (1995)|
|1488.0   |450   |5.02150